# Simulation of a Ball's Descend in a Terrain

This project simulates where a ball will land in a terrain.

## Input
The terrain's configuration is given as a matrix of integers representing elevation at each spot. For simplicity, assume that the terrain is surrounded by a rectangular wall, that prevents the ball to escape. The inner dimensions of the terrain are NxM, where N and M are integers between 3 and 1000.

The ball's initial position is given as a pair of integers (a, b).

## Output
The result is a list of coordinates denoting the ball's path in a terrain. The first element of the list is the starting position, and the last one is the ending position. It could happen that they are the same, if the ball has emanated from a local minima (dent).

## Rules
The ball moves according to the next two simple rules:
- The ball rolls from the current position into the lowest neighboring one.
- If the ball is surrounded by higher points, then it stops.

In [1]:
# Usual bootstrapping code; just run this cell.
import numpy as np

from typing import List, Tuple

from ipywidgets import interact, widgets

In [2]:
terrain = np.matrix([
    [-2, 3, 2, 1],
    [-2, 4, 3, 0],
    [-3, 3, 1, -3],
    [-4, 2, -1, 1],
    [-5, -7, 3, 0]
])
terrain

matrix([[-2,  3,  2,  1],
        [-2,  4,  3,  0],
        [-3,  3,  1, -3],
        [-4,  2, -1,  1],
        [-5, -7,  3,  0]])

In [3]:
def wall(terrain:np.matrix, position:Tuple[int,int]) -> bool:
    """
    Checks whether the provided position is hitting the wall.
    
    Args:
    terrain: the terrain's configuration comprised from integer elevation levels.
    position: the pair of integers representing the ball's potential position.

    Output:
    True if the position is hitting the wall, or False otherwise.
    
    Examples:
    >>> wall(np.matrix([[-2, 3, 2, 1]]), (0, 1))
    False
    >>> wall(np.matrix([[-2, 3, 2, 1]]), (-1, 0))
    True
    """
    
    x, y = position
    length, width = terrain.shape
    return (x < 0) or (y < 0) or (x >= length) or (y >= width)

def next_neighbor(terrain:np.matrix, position:Tuple[int,int]) -> Tuple[int,int]:
    """
    Returns the position of the lowest neighbor.
    
    Args:
    terrain: the terrain's configuration comprised from integer elevation levels.
    position: the pair of integers representing the ball's current position.

    Output:
    The position (pair of coordinates) of the lowest neighbor.
    
    Example:
    >>> next_neighbor(np.matrix([[-2, 3, 2, 1]]), (0, 1))
    (0, 0)
    """
    
    x, y = position
    allowed_neighbors = []
    for delta_x in range(-1, 2):
        for delta_y in range(-1, 2):
            new_position = (x + delta_x, y + delta_y)
            if (not wall(terrain, new_position)):
                allowed_neighbors.append((terrain.item(new_position), new_position))
    return min(allowed_neighbors)[1]

def find_path(terrain:np.matrix, position:Tuple[int,int]) -> List[Tuple[int,int]]:
    """
    Find the path that the ball would follow while descending in the terrain.
    
    Args:
    terrain: the terrain's configuration comprised from integer elevation levels.
    position: the pair of integers representing the ball's current position.
    
    Output:
    The list of coordinates of the path.
    
    Example:
    >>> find_path(np.matrix([[-2, 3, 2, 1]]), (0, 1))
    [(0, 1), (0, 0)]
    """
    
    next_position = next_neighbor(terrain, position)
    if (position == next_position):
        return [position]
    else:
        return [position] + find_path(terrain, next_position)

## Result

The cell below contains code to invoke the path finding function for a given starting position. The starting coordinates are expected to be correctly set.

The terrain data is repeated here for convenience.

In [4]:
terrain

matrix([[-2,  3,  2,  1],
        [-2,  4,  3,  0],
        [-3,  3,  1, -3],
        [-4,  2, -1,  1],
        [-5, -7,  3,  0]])

In [7]:
interact(lambda start_x, start_y: find_path(terrain, (start_x, start_y)),
         start_x = widgets.IntSlider(value=1, max=terrain.shape[0]-1, description='Start X'),
         start_y = widgets.IntSlider(value=1, max=terrain.shape[1]-1, description='Start Y'));

interactive(children=(IntSlider(value=1, description='Start X', max=4), IntSlider(value=1, description='Start …

In [6]:
# Just run this cell to invoke tests embedded inside function descriptors.
import doctest
doctest.testmod(verbose=True)

Trying:
    find_path(np.matrix([[-2, 3, 2, 1]]), (0, 1))
Expecting:
    [(0, 1), (0, 0)]
ok
Trying:
    next_neighbor(np.matrix([[-2, 3, 2, 1]]), (0, 1))
Expecting:
    (0, 0)
ok
Trying:
    wall(np.matrix([[-2, 3, 2, 1]]), (0, 1))
Expecting:
    False
ok
Trying:
    wall(np.matrix([[-2, 3, 2, 1]]), (-1, 0))
Expecting:
    True
ok
1 items had no tests:
    __main__
3 items passed all tests:
   1 tests in __main__.find_path
   1 tests in __main__.next_neighbor
   2 tests in __main__.wall
4 tests in 4 items.
4 passed and 0 failed.
Test passed.


TestResults(failed=0, attempted=4)